In [2]:
import logging
import os
from datetime import datetime
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, GRU, LSTM, MultiHeadAttention
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Constants
CONFIG = {
    'forecast_horizon': 12,
    'seasonal_periods': 12,
    'min_data_length': 50,
    'img_dir': 'dl_model_results',
    'results_file': 'dl_model_results/deep_learning_model_results.csv',
    'lags': list(range(1, 13)),
    'rolling_windows': [3, 6, 12],
    'timesteps': 3,
    'batch_size': 32,
    'epochs': 100,
    'early_stopping_patience': 10,
    'reduce_lr_patience': 5,
    'reduce_lr_factor': 0.2,
    'learning_rate': 0.005,
    'k_folds': 5,  # Added for K-fold cross-validation
    'model_params': {
        'LSTM': {'units': 16, 'type': 'lstm'},  # Increased units for better capacity
        'GRU': {'units': 16, 'type': 'gru'},
        'CNN-LSTM': {'filters': 16, 'kernel_size': 3, 'units': 32, 'type': 'cnn_lstm'},
        'Transformer': {'num_heads': 4, 'key_dim': 16, 'dense_units': 32, 'type': 'transformer'},
        'TCN': {'filters': 16, 'kernel_size': 3, 'type': 'tcn'}
    }
}

# Logging setup
os.makedirs(CONFIG['img_dir'], exist_ok=True)
logging.basicConfig(
    filename=os.path.join(CONFIG['img_dir'], 'dl_models_log.txt'),
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def validate_data(data: pd.DataFrame, required_columns: List[str] = ['cpi', 'time', 'oil_price', 'gold_price']) -> None:
    """Validate input data for required columns, length, and data types."""
    logger.info("Validating input data")
    missing_cols = [col for col in required_columns if col not in data.columns]
    if missing_cols:
        logger.error(f"Missing columns: {missing_cols}")
        raise ValueError(f"Missing columns: {missing_cols}")

    if len(data) < CONFIG['min_data_length'] + CONFIG['forecast_horizon'] * 2:
        logger.error(f"Data too short: {len(data)} rows")
        raise ValueError(f"Data too short: {len(data)} rows")

    numeric_cols = [col for col in required_columns if col != 'time']
    for col in numeric_cols:
        if not pd.api.types.is_numeric_dtype(data[col]):
            logger.error(f"Column {col} is not numeric: {data[col].dtype}")
            raise ValueError(f"Column {col} is not numeric: {data[col].dtype}")
        data[col] = pd.to_numeric(data[col], errors='coerce')

    if data[numeric_cols].isna().any().any():
        logger.error(f"Data contains NaN: {data[numeric_cols].isna().sum().to_dict()}")
        raise ValueError("Data contains NaN")
    if np.isinf(data[numeric_cols]).any().any():
        logger.error(f"Data contains Inf: {np.isinf(data[numeric_cols]).sum().to_dict()}")
        raise ValueError("Data contains Inf")

    logger.info(f"Data valid: {len(data)} rows, columns: {list(data.columns)}")

def create_features(
    data: pd.DataFrame,
    target: str,
    lags: List[int] = CONFIG['lags'],
    rolling_windows: List[int] = CONFIG['rolling_windows'],
    trend_features: bool = True,
    seasonal_features: bool = True,
    fill_method: str = 'interpolate',
    correlation_threshold: float = 0.2,
    use_pca: bool = False,
    pca_variance_ratio: float = 0.95
) -> pd.DataFrame:
    """Generate features for time series data."""
    logger.info(f"Generating features for {target}")
    if not isinstance(data.index, pd.DatetimeIndex):
        logger.error("Index must be DatetimeIndex")
        raise ValueError("Index must be DatetimeIndex")

    df = data.copy()
    exog_vars = ['oil_price', 'gold_price']
    required_cols = [target] + exog_vars
    if not all(col in df.columns for col in required_cols):
        logger.error(f"Missing required columns: {required_cols}")
        raise ValueError(f"Missing required columns: {required_cols}")

    for col in required_cols:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
        for window in rolling_windows:
            df[f'{col}_roll_mean_{window}'] = df[col].rolling(window=window).mean()
            df[f'{col}_roll_std_{window}'] = df[col].rolling(window=window).std()

    if trend_features:
        df[f'{target}_diff_1'] = df[target].diff()
        df[f'{target}_trend'] = np.arange(len(df))

    if seasonal_features:
        period = CONFIG['seasonal_periods']
        df['month'] = df.index.month
        df = pd.get_dummies(df, columns=['month'], prefix='month')
        df['month_sin'] = np.sin(2 * np.pi * df.index.month / period)
        df['month_cos'] = np.cos(2 * np.pi * df.index.month / period)

    df['quarter'] = df.index.quarter
    df['year'] = df.index.year

    fill_methods = {
        'interpolate': lambda x: x.interpolate(method='linear', limit_direction='both'),
        'rolling_mean': lambda x: x.fillna(x.rolling(window=3, min_periods=1).mean()),
        'ffill': lambda x: x.ffill(),
        'bfill': lambda x: x.bfill()
    }
    for col in df.columns:
        if df[col].isna().any():
            df[col] = fill_methods.get(fill_method, fill_methods['interpolate'])(df[col])
            df[col] = df[col].fillna(df[col].mean())

    if df.isna().any().any():
        logger.error(f"Data still contains NaN: {df.isna().sum().to_dict()}")
        raise ValueError("Data still contains NaN")

    if correlation_threshold > 0:
        correlations = df.corr()[target].drop(required_cols)
        low_corr_cols = correlations[abs(correlations) < correlation_threshold].index
        df.drop(columns=low_corr_cols, inplace=True)

    if use_pca and len(df.columns) > len(required_cols):
        feature_cols = [col for col in df.columns if col not in required_cols]
        if feature_cols:
            pca = PCA(n_components=pca_variance_ratio, svd_solver='full')
            pca_features = pca.fit_transform(df[feature_cols])
            pca_cols = [f'pca_{i+1}' for i in range(pca.n_components_)]
            df = pd.concat([df[required_cols], pd.DataFrame(pca_features, index=df.index, columns=pca_cols)], axis=1)

    logger.info(f"Processed features: {list(df.columns)}")
    return df

def calculate_metrics(actual: np.ndarray, predicted: np.ndarray) -> Tuple[float, ...]:
    """Calculate evaluation metrics."""
    actual, predicted = np.array(actual, dtype=float), np.array(predicted, dtype=float)
    valid_mask = ~(np.isnan(actual) | np.isnan(predicted) | np.isinf(actual) | np.isinf(predicted))
    actual, predicted = actual[valid_mask], predicted[valid_mask]

    if len(actual) == 0:
        logger.warning("No valid data for metrics calculation")
        return tuple([np.nan] * 6)

    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    mape = mean_absolute_percentage_error(actual, predicted) * 100 if np.all(np.abs(actual) > 1e-8) else np.nan
    smape = 100 * np.mean(2 * np.abs(predicted - actual) / (np.abs(actual) + np.abs(predicted)))
    norm_mape = mape / np.mean(np.abs(actual)) if not np.isnan(mape) else np.nan
    directional_acc = np.mean((np.diff(actual) * np.diff(predicted)) > 0) * 100 if len(actual) > 1 else np.nan

    return rmse, mae, mape, smape, norm_mape, directional_acc

def plot_forecast(
    historical: pd.Series,
    val: pd.Series,
    test: pd.Series,
    forecast: pd.Series,
    forecast_index: pd.DatetimeIndex,
    title: str,
    ylabel: str,
    filename: str
) -> None:
    """Plot forecast results."""
    plt.figure(figsize=(12, 6))
    plt.plot(historical.index, historical, label='Historical', color='blue')
    plt.plot(val.index, val, label='Validation', color='purple')
    plt.plot(test.index, test, label='Actual', color='green')
    plt.plot(forecast_index[:len(forecast)], forecast, label='Forecast', color='orange', linestyle='--')
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['img_dir'], filename))
    plt.close()
    logger.info(f"Saved plot: {filename}")

def plot_comparison_forecasts(
    historical: pd.Series,
    val: pd.Series,
    test: pd.Series,
    forecasts: Dict[str, pd.Series],
    forecast_index: pd.DatetimeIndex,
    title: str,
    ylabel: str,
    filename: str,
    metrics: Dict[str, Dict[str, float]]
) -> None:
    """Plot comparison of forecasts from multiple models."""
    plt.figure(figsize=(14, 8))
    plt.plot(historical.index, historical, label='Historical', color='blue')
    plt.plot(val.index, val, label='Validation', color='purple')
    plt.plot(test.index, test, label='Actual', color='green')
    colors = sns.color_palette("husl", len(forecasts))
    for (model_name, forecast), color in zip(forecasts.items(), colors):
        rmse = metrics.get(model_name, {}).get('Test RMSE', np.nan)
        if pd.isna(rmse) or forecast is None:
            continue
        plt.plot(forecast_index[:len(forecast)], forecast, label=f'{model_name} (RMSE: {rmse:.4f})',
                 linestyle='--', color=color)
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG['img_dir'], filename))
    plt.close()
    logger.info(f"Saved comparison plot: {filename}")

def prepare_data_for_dl(
    train: pd.DataFrame,
    val: pd.DataFrame,
    test: pd.DataFrame,
    target: str,
    feature_cols: List[str],
    timesteps: int = CONFIG['timesteps']
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, RobustScaler]:
    """Prepare data for deep learning models."""
    scaler = RobustScaler()
    X_train = scaler.fit_transform(train[feature_cols])
    X_val = scaler.transform(val[feature_cols])
    X_test = scaler.transform(test[feature_cols])
    y_train = scaler.fit_transform(train[[target]]).flatten()
    y_val = scaler.transform(val[[target]]).flatten()
    y_test = scaler.transform(test[[target]]).flatten()

    def create_sequences(X: np.ndarray, y: np.ndarray, timesteps: int) -> Tuple[np.ndarray, np.ndarray]:
        X_seq, y_seq = [], []
        for i in range(len(X) - timesteps):
            X_seq.append(X[i:i + timesteps])
            y_seq.append(y[i + timesteps])
        return np.array(X_seq), np.array(y_seq)

    X_train_seq, y_train_seq = create_sequences(X_train, y_train, timesteps)
    X_val_seq, y_val_seq = create_sequences(X_val, y_val, timesteps)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, timesteps)

    logger.info(f"Data shapes: Train={X_train_seq.shape}, Val={X_val_seq.shape}, Test={X_test_seq.shape}")
    return X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, scaler

def build_model(model_type: str, input_shape: Tuple[int, int], params: Dict[str, any]) -> Model:
    """Build a deep learning model based on type with enhanced architecture."""
    if model_type == 'transformer':
        inputs = tf.keras.Input(shape=input_shape)
        x = MultiHeadAttention(num_heads=params['num_heads'], key_dim=params['key_dim'],
                               kernel_regularizer=l2(0.01))(inputs, inputs)
        x = Dropout(0.4)(x)
        x = Flatten()(x)
        x = Dense(params['dense_units'], activation='relu')(x)
        outputs = Dense(1)(x)
        return tf.keras.Model(inputs, outputs)

    model = Sequential()
    if model_type == 'lstm':
        model.add(LSTM(params['units'], input_shape=input_shape, return_sequences=True, kernel_regularizer=l2(0.01)))
        model.add(LSTM(params['units'] // 2, kernel_regularizer=l2(0.01)))
    elif model_type == 'gru':
        model.add(GRU(params['units'], input_shape=input_shape, return_sequences=True, kernel_regularizer=l2(0.01)))
        model.add(GRU(params['units'] // 2, kernel_regularizer=l2(0.01)))
    elif model_type == 'cnn_lstm':
        model.add(Conv1D(params['filters'], kernel_size=params['kernel_size'], activation='relu',
                         input_shape=input_shape, padding='same', kernel_regularizer=l2(0.01)))
        model.add(tf.keras.layers.MaxPooling1D(2))
        model.add(LSTM(params['units'], kernel_regularizer=l2(0.01)))
    elif model_type == 'tcn':
        model.add(Conv1D(params['filters'], kernel_size=params['kernel_size'], activation='relu',
                         input_shape=input_shape, padding='causal', kernel_regularizer=l2(0.01)))
        model.add(Conv1D(params['filters'] // 2, kernel_size=params['kernel_size'], activation='relu',
                         padding='causal', kernel_regularizer=l2(0.01)))
        model.add(Flatten())

    model.add(Dropout(0.5))
    model.add(Dense(1))
    return model

def run_model(
    train: pd.DataFrame,
    val: pd.DataFrame,
    test: pd.DataFrame,
    forecast_index: pd.DatetimeIndex,
    target: str,
    feature_cols: List[str],
    model_name: str,
    params: Dict[str, any]
) -> Optional[Dict[str, any]]:
    """Run a deep learning model with K-fold cross-validation and ensemble forecasting."""
    start_time = datetime.now()
    try:
        X_train_seq, y_train_seq, X_val_seq, y_val_seq, X_test_seq, y_test_seq, scaler = prepare_data_for_dl(
            train, val, test, target, feature_cols
        )

        # K-fold cross-validation
        tscv = TimeSeriesSplit(n_splits=CONFIG['k_folds'])
        val_rmses = []
        for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train_seq)):
            X_fold_train, y_fold_train = X_train_seq[train_idx], y_train_seq[train_idx]
            X_fold_val, y_fold_val = X_train_seq[val_idx], y_train_seq[val_idx]

            model = build_model(params['type'], (CONFIG['timesteps'], X_train_seq.shape[2]), params)
            model.compile(optimizer=Adam(learning_rate=CONFIG['learning_rate']), loss='mse')

            callbacks = [
                EarlyStopping(patience=CONFIG['early_stopping_patience'], restore_best_weights=True, monitor='val_loss'),
                ReduceLROnPlateau(factor=CONFIG['reduce_lr_factor'], patience=CONFIG['reduce_lr_patience'],
                                 monitor='val_loss', min_lr=1e-6)
            ]

            model.fit(
                X_fold_train, y_fold_train,
                validation_data=(X_fold_val, y_fold_val),
                epochs=CONFIG['epochs'],
                batch_size=CONFIG['batch_size'],
                callbacks=callbacks,
                verbose=0
            )

            val_pred = scaler.inverse_transform(model.predict(X_fold_val, verbose=0)).flatten()
            val_true = scaler.inverse_transform(y_fold_val.reshape(-1, 1)).flatten()
            val_rmse = np.sqrt(mean_squared_error(val_true, val_pred))
            val_rmses.append(val_rmse)
            logger.info(f"{model_name} ({target}) Fold {fold+1}: Val RMSE={val_rmse:.4f}")

        avg_val_rmse = np.mean(val_rmses)
        logger.info(f"{model_name} ({target}) Average Val RMSE across {CONFIG['k_folds']} folds: {avg_val_rmse:.4f}")

        # Train final model on full training data
        model = build_model(params['type'], (CONFIG['timesteps'], X_train_seq.shape[2]), params)
        model.compile(optimizer=Adam(learning_rate=CONFIG['learning_rate']), loss='mse')
        model.fit(
            X_train_seq, y_train_seq,
            validation_data=(X_val_seq, y_val_seq),
            epochs=CONFIG['epochs'],
            batch_size=CONFIG['batch_size'],
            callbacks=callbacks,
            verbose=0
        )

        # Ensemble forecasting (average predictions over multiple runs)
        n_ensemble = 3
        forecasts_scaled = []
        for _ in range(n_ensemble):
            input_seq = X_test_seq[-1].copy()
            forecast_scaled = []
            for i in range(CONFIG['forecast_horizon']):
                pred = model.predict(input_seq.reshape(1, CONFIG['timesteps'], X_train_seq.shape[2]), verbose=0)
                forecast_scaled.append(pred[0, 0])
                input_seq = np.roll(input_seq, -1, axis=0)
                input_seq[-1, feature_cols.index(f'{target}_lag_1')] = y_test_seq[i] if i < len(y_test_seq) else pred[0, 0]
            forecasts_scaled.append(forecast_scaled)
        
        forecast_scaled = np.mean(forecasts_scaled, axis=0)
        forecast = scaler.inverse_transform(np.array(forecast_scaled).reshape(-1, 1)).flatten()
        forecast = pd.Series(forecast, index=forecast_index)

        train_pred = scaler.inverse_transform(model.predict(X_train_seq, verbose=0)).flatten()
        val_pred = scaler.inverse_transform(model.predict(X_val_seq, verbose=0)).flatten()
        train_metrics = calculate_metrics(scaler.inverse_transform(y_train_seq.reshape(-1, 1)).flatten(), train_pred)
        val_metrics = calculate_metrics(scaler.inverse_transform(y_val_seq.reshape(-1, 1)).flatten(), val_pred)
        test_metrics = calculate_metrics(scaler.inverse_transform(y_test_seq.reshape(-1, 1)).flatten(),
                                        forecast[:len(y_test_seq)])

        plot_forecast(
            train[target][-48:], val[target], test[target][:len(forecast)],
            forecast, forecast_index, f'{model_name} Forecast for {target}',
            target, f'{target}_{model_name.lower().replace("-", "_")}_forecast.png'
        )

        model.save(os.path.join(CONFIG['img_dir'], f'{model_name.lower().replace("-", "_")}_{target}.h5'))

        elapsed_time = (datetime.now() - start_time).total_seconds()
        logger.info(f"{model_name} ({target}): Train RMSE={train_metrics[0]:.4f}, "
                    f"Val RMSE={val_metrics[0]:.4f}, Test RMSE={test_metrics[0]:.4f}, Time={elapsed_time:.2f}s")

        return {
            'forecast': forecast,
            'residuals': scaler.inverse_transform(y_train_seq.reshape(-1, 1)).flatten() - train_pred,
            'train_rmse': train_metrics[0],
            'val_rmse': val_metrics[0],
            'test_rmse': test_metrics[0],
            'test_mae': test_metrics[1],
            'test_mape': test_metrics[2],
            'test_smape': test_metrics[3],
            'test_norm_mape': test_metrics[4],
            'test_dir_acc': test_metrics[5]
        }
    except Exception as e:
        logger.error(f"Error in {model_name} ({target}): {str(e)}")
        return None

def main() -> None:
    """Main program to run deep learning models for time series forecasting."""
    tf.keras.utils.set_random_seed(42)
    try:
        data = pd.read_csv('data/data.csv')
        validate_data(data)

        data['time'] = pd.to_datetime(data['time'])
        data.set_index('time', inplace=True)

        features_cpi = create_features(data, 'cpi', trend_features=False)
        train_cpi = features_cpi[:-36]
        val_cpi = features_cpi[-36:-12]
        test_cpi = features_cpi[-12:]

        forecast_index = pd.date_range(start=test_cpi.index[0], periods=CONFIG['forecast_horizon'], freq='MS')
        feature_cols_cpi = [col for col in train_cpi.columns if col != 'cpi']

        results, forecasts_cpi, metrics_cpi = [], {}, {}
        for model_name, params in CONFIG['model_params'].items():
            result = run_model(
                train_cpi, val_cpi, test_cpi, forecast_index, 'cpi',
                feature_cols_cpi, model_name, params
            )
            if result:
                results.append({
                    'Target': 'cpi',
                    'Model': model_name,
                    'Train RMSE': result['train_rmse'],
                    'Val RMSE': result['val_rmse'],
                    'Test RMSE': result['test_rmse'],
                    'Test MAE': result['test_mae'],
                    'Test MAPE': result['test_mape'],
                    'Test sMAPE': result['test_smape'],
                    'Test NormMAPE': result['test_norm_mape'],
                    'Test DirAcc': result['test_dir_acc']
                })
                forecasts_cpi[model_name] = result['forecast']
                metrics_cpi[model_name] = {'Test RMSE': result['test_rmse']}

        if forecasts_cpi:
            # Ensemble forecast by averaging
            valid_forecasts = [f for f in forecasts_cpi.values() if f is not None]
            if valid_forecasts:
                ensemble_forecast = pd.concat(valid_forecasts, axis=1).mean(axis=1)
                ensemble_forecast = pd.Series(ensemble_forecast, index=forecast_index)
                forecasts_cpi['Ensemble'] = ensemble_forecast
                metrics_cpi['Ensemble'] = {
                    'Test RMSE': np.sqrt(mean_squared_error(test_cpi['cpi'][:len(ensemble_forecast)],
                                                          ensemble_forecast[:len(test_cpi['cpi'])]))
                }

                plot_forecast(
                    train_cpi['cpi'][-48:], val_cpi['cpi'], test_cpi['cpi'],
                    ensemble_forecast, forecast_index, 'Ensemble Forecast for CPI',
                    'CPI', 'cpi_ensemble_forecast.png'
                )

            plot_comparison_forecasts(
                train_cpi['cpi'][-36:], val_cpi['cpi'], test_cpi['cpi'],
                forecasts_cpi, forecast_index, 'DL Model Comparison for CPI',
                'CPI', 'cpi_dl_model_comparison.png', metrics_cpi
            )

        results_df = pd.DataFrame(results)
        print("Model Results:")
        print(results_df)
        results_df.to_csv(CONFIG['results_file'], index=False)
        logger.info(f"Results saved to {CONFIG['results_file']}")

        if forecasts_cpi:
            combined_forecast = pd.DataFrame({'Date': forecast_index})
            for model_name, forecast in forecasts_cpi.items():
                combined_forecast[f'{model_name}_cpi'] = forecast
            combined_forecast.to_csv(os.path.join(CONFIG['img_dir'], 'combined_forecast_cpi.csv'), index=False)
            logger.info(f"Combined forecast saved to {CONFIG['img_dir']}/combined_forecast_cpi.csv")

    except Exception as e:
        logger.error(f"Main program error: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Model Results:
  Target        Model  Train RMSE  Val RMSE  Test RMSE  Test MAE  Test MAPE  \
0    cpi         LSTM    0.488614  0.292663   0.161518  0.124461   0.124329   
1    cpi          GRU    0.534295  0.306378   0.201700  0.157457   0.157295   
2    cpi     CNN-LSTM    0.488801  0.286947   0.131971  0.111849   0.111583   
3    cpi  Transformer    0.516502  0.289119   0.154192  0.134223   0.133980   
4    cpi          TCN    0.712442  0.319567   0.147439  0.132742   0.132483   

   Test sMAPE  Test NormMAPE  Test DirAcc  
0    0.124204       0.001241        100.0  
1    0.157104       0.001570         62.5  
2    0.111622       0.001114        100.0  
3    0.133940       0.001337         37.5  
4    0.132465       0.001322         62.5  
